#### Includes

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import altair as alt
from altair import datum
import pandas as pd
import time

In [3]:
import sources
from src.config.attribute_dict import AttributeDict
from src.config import parse_config
from steered_cnn.models import *
from steered_cnn.steered_conv import SteeredConvBN
from steered_cnn.utils import ConvBN

## Init

In [4]:
cfg = """
experiment:
    name: 'test'
    sub-experiment: 'test'

model:
    backbone: 'hemeling'
    steered:
        steering: 'angle'
    nfeatures: 11
    nscale: 5
    depth: 2
    kernel: 5

hyper-parameters:
    batch-size: 4
"""


cfg = parse_config(cfg)

In [5]:
train, val, test = sources.load_dataset(cfg)

In [6]:
net = setup_model(cfg.model, 6, 1).cuda()
net

SteeredHemelingNet(
  (downconv0-0): SteeredConvBN(
    (conv): SteeredConv2d()
    (bn_relu): Sequential(
      (0): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
    )
  )
  (downconv0-1): SteeredConvBN(
    (conv): SteeredConv2d()
    (bn_relu): Sequential(
      (0): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
    )
  )
  (downsample0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (downconv1-0): SteeredConvBN(
    (conv): SteeredConv2d()
    (bn_relu): Sequential(
      (0): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
    )
  )
  (downconv1-1): SteeredConvBN(
    (conv): SteeredConv2d()
    (bn_relu): Sequential(
      (0): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
    )
  )
  (downsample1): MaxPool2d(kernel_size=2, stride=2, padding=0, d

## Run Model

In [7]:
for data in train:
    break

### Monitor Shapes

In [8]:
def hook(name):
    def print_shape(mod, inputs, outputs):
        print(name, ': ', [_.shape for _ in inputs], ' -> ' , outputs.shape)
    return print_shape
for name, mod in net.named_modules():
    if isinstance(mod, (SteeredConvBN, ConvBN)):
        mod.register_forward_hook(hook(name))

In [9]:
args = {'x': data['x'].cuda()}
if 'alpha' in data:
    args['alpha'] = data['alpha'].cuda()

r = net(**args)
print(data['x'].shape, ' -> ' ,r.shape)

downconv0-0 :  [torch.Size([4, 6, 565, 565])]  ->  torch.Size([4, 11, 565, 565])
downconv0-1 :  [torch.Size([4, 11, 565, 565])]  ->  torch.Size([4, 11, 565, 565])
downconv1-0 :  [torch.Size([4, 11, 282, 282])]  ->  torch.Size([4, 22, 282, 282])
downconv1-1 :  [torch.Size([4, 22, 282, 282])]  ->  torch.Size([4, 22, 282, 282])
downconv2-0 :  [torch.Size([4, 22, 141, 141])]  ->  torch.Size([4, 44, 141, 141])
downconv2-1 :  [torch.Size([4, 44, 141, 141])]  ->  torch.Size([4, 44, 141, 141])
downconv3-0 :  [torch.Size([4, 44, 70, 70])]  ->  torch.Size([4, 88, 70, 70])
downconv3-1 :  [torch.Size([4, 88, 70, 70])]  ->  torch.Size([4, 88, 70, 70])
downconv4-0 :  [torch.Size([4, 88, 35, 35])]  ->  torch.Size([4, 176, 35, 35])
downconv4-1 :  [torch.Size([4, 176, 35, 35])]  ->  torch.Size([4, 176, 35, 35])
upconv5-0 :  [torch.Size([4, 176, 68, 68])]  ->  torch.Size([4, 88, 68, 68])
upconv5-1 :  [torch.Size([4, 88, 68, 68])]  ->  torch.Size([4, 88, 68, 68])
upconv6-0 :  [torch.Size([4, 88, 134, 134